# Probabilidades e análise de texto

Esta é uma prévia para o Projeto 1.

Leia [esta referência do Prof. Sebastian Raschka](https://arxiv.org/pdf/1410.5329.pdf) após concluir a atividade. Também existe [em formato de blog](https://sebastianraschka.com/Articles/2014_naive_bayes_1.html)

A técnica apresentada neste notebook foi usada para resolver [um problema histórico de autoria de documentos](https://priceonomics.com/how-statistics-solved-a-175-year-old-mystery-about/).

## Índice

- [Análise "Os Lusíadas" vs "Dom Casmurro"](#analise-lusiadas-vs-dom-casmurro)
    - [Limpeza das bases](#limpeza-das-bases)
    - [Tabelas de frequências: Dom Casmurro](#tabelas-de-frequencias-dom-casmurro)
        - [Frequências absolutas](#frequencias-absolutas-casmurro)
        - [Frequências relativas](#frequencias-relativas-casmurro)
    - [Tabelas de frequências: Os Lusíadas](#tabelas-de-frequencias-os-lusiadas)
- [Probabilidades](#probabilidades)
- [Probabilidades na língua portuguesa](#probabilidades-na-lingua-portuguesa)
- [Classificação: Lusíadas ou Dom Casmurro?](#classificacao-lusiadas-ou-dom-casmurro)
    - [A ingenuidade - Naïve Bayes](#a-ingenuidade-naive-bayes)

- [O que você deve fazer](#o-que-voce-deve-fazer)
    - [Exercício 1](#ex1)
    - [Exercício 2](#ex2)
    - [Exercício 3](#ex3)
    - [Exercício 4](#ex4)
- [O que você deve pesquisar para fazer o Projeto 1](#o-que-pesquisar)



In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 13

Abaixo definimos uma função de limpeza simples, que usaremos nos trechos de texto analizados

In [2]:
import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
    

<div id="analise-lusiadas-vs-dom-casmurro"></div>

## Análise "Os Lusíadas" vs "Dom Casmurro"

Vamos analisar as obras Dom Casmurro, de [Machado de Assis](http://machado.mec.gov.br/) e Os Lusíadas, de [Luis Vaz de Camões](https://pt.wikipedia.org/wiki/Lu%C3%ADs_de_Cam%C3%B5es). Ambas as obras são de domínio público e foram obtidas no site do [Projeto Gutenberg](https://www.gutenberg.org/wiki/PT_Principal).

Nosso objetivo é: dada uma frase extraída de alguma das duas obras, identificar a qual obra a frase pertence.
Ou seja, queremos classificar uma frase como pertencendo a "Os Lusíadas" ou a "Dom Casmurro". 

Para isso temos somente a informação de quais **palavras constituem a frase**. Como podemos utilizar as palavras para descobrir a qual obra a frase pertence? Você deve ter algum conhecido que sempre usa alguma palavra um pouco estranha ou diferente que somente ele/ela usa. A intuição no nosso caso é semelhante: cada autor deve ter algumas palavras específicas que usa com mais frequência do que o outro autor. Se isso for verdade, poderíamos tentar descobrir qual é a obra dependendo de quais palavras aparecem na frase.

Vamos pensar em termos de probabilidade. A informação que temos disponível sobre a frase são as palavras. Assim, temos os seguintes eventos:

- $Casmurro$: a frase foi extraída da obra "Dom Casmurro";
- $Lusíadas$: a frase foi extraída da obra "Os Lusíadas";
- $frase$: uma determinada frase foi encontrada.

Utilizamos $w_i$ como um nome genérico para qualquer palavra utilizada em algum dos textos.

A partir das palavras contidas na frase, queremos calcular a probabilidade da frase ter sido extraída de "Dom Casmurro" ou de "Os Lusíadas". Traduzindo em termos de probabilidades:

- $P(Casmurro|frase)$: probabilidade da obra original ser "Dom Casmurro" dado a frase;
- $P(Lusíadas|frase)$: probabilidade da obra original ser "Os Lusíadas" dado a frase.

Se $P(Casmurro|frase) > P(Lusíadas|frase)$ classificaremos a frase como extraída de "Dom Casmurro", senão, classificaremos como "Os Lusíadas". Mas como podemos calcular essas probabilidades?

Vimos na atividade da aula (dataset do Titanic), que é possível utilizar a regra de Bayes para obter uma probabilidade condicional da seguinte forma (vamos utilizar $P(Casmurro|frase)$ como exemplo, mas o processo é análogo para $P(Lusíadas|frase)$:

$$P(Casmurro|frase) = \frac{P(frase|Casmurro) P(Casmurro)}{P(frase)}$$

Ou seja, precisamos de: 
- $P(frase|Casmurro)$: probabilidade de encontrar a frase na obra "Dom Casmurro";
- $P(frase|Lusíadas)$: probabilidade de encontrar a frase na obra "Os Lusíadas";
- $P(Casmurro)$: probabilidade da obra original ser "Dom Casmurro";
- $P(Lusíadas)$: probabilidade da obra original ser "Os Lusíadas";
- $P(frase)$: probabilidade de uma frase ocorrer na língua portuguesa.

Veremos que temos como conseguir estimativas de todas essas probabilidades. Assim poderemos obter as probabilidades desejadas. 

**Observação**: classificadores como o que vamos construir, que utilizam a regra de Bayes para calcular as probabilidades, são chamados Classificadores Bayesianos. Na verdade, vamos utilizar uma simplificação, conhecida como **Naïve Bayes**.

Vamos começar carregando o texto de cada obra.

In [3]:
# Carrega o arquivo de texto
# Como ele possui caracteres com acentos, precisamos carregar utilizando o encoding="utf8".
with open("textos/lusiadas_texto.txt", "r", encoding="utf-8-sig") as arquivo_texto:
    lusiadas_raw = arquivo_texto.read()

# Imprime os 100 primeiros caracteres do texto para verificar se está tudo ok.
print(lusiadas_raw[0:100])

Luís Vaz de Camões
Os Lusíadas

Canto Primeiro

1
As armas e os barões assinalados,
Que da ocidental


In [4]:
with open("textos/domcasmurro_texto.txt", "r", encoding="utf-8-sig") as arquivo_texto:
    casmurro_raw = arquivo_texto.read()
    
print(casmurro_raw[0:100])

DOM CASMURRO

POR

MACHADO DE ASSIS

DA ACADEMIA BRAZILEIRA

H. GARNIER, LIVREIRO-EDITOR

RUA MOREIR


<div id="limpeza-das-bases"></div>

### Limpezas das bases

Em nossa análise será necessário calcular a frequência das palavras no texto. Entretanto, o Python diferencia caracteres maiúsculos (e.g. `'A'` ou `'B'`) de minúsculos (e.g. `'a'` e `'b'`). Por isso, vamos converter todo o texto para letras minúsculas.

Além disso, apesar de serem importantes para a língua, vamos simplificar nossa análise removendo sinais de pontuação. Para isso vamos aplicar uma limpeza rudimentar com a função `cleanup()` definida anteriormente.

In [5]:
lusiadas = cleanup(lusiadas_raw.lower())

In [6]:
casmurro = cleanup(casmurro_raw.lower())

Vamos inspecionar  os arquivos

In [7]:
print(lusiadas[0:100])

luís vaz de camões
os lusíadas

canto primeiro

1
as armas e os barões assinalados 
que da ocidental


In [8]:
print(casmurro[0:100])

dom casmurro

por

machado de assis

da academia brazileira

h  garnier  livreiro editor

rua moreir


<div id="tabelas-de-frequencias-dom-casmurro"></div>

### Tabelas de frequências: Dom Casmurro

Vamos começar contando as palavras em cada texto, ou seja, vamos construir a tabela de frequência absoluta para as palavras de cada obra. Primeiramente vamos converter as obras (strings contendo o texto completo) em objetos do tipo `pd.Series`, nos quais cada elemento é uma palavra.

In [9]:
todas_palavras_casmurro = casmurro.split()
todas_palavras_casmurro[0:10]

['dom',
 'casmurro',
 'por',
 'machado',
 'de',
 'assis',
 'da',
 'academia',
 'brazileira',
 'h']

In [10]:
serie_casmurro = pd.Series(todas_palavras_casmurro)
serie_casmurro

0             dom
1        casmurro
2             por
3         machado
4              de
5           assis
           ...   
67472     cxlviii
67473           é
67474         bem
67475           e
67476           o
67477       resto
Length: 67478, dtype: object

In [11]:
serie_casmurro.head()

0         dom
1    casmurro
2         por
3     machado
4          de
dtype: object

Palavras em um texto são variáveis **qualitativas nominais**, portanto usaremos `value_counts()` para obter a tabela de frequências relativas e absolutas:

<div id="frequencias-absolutas-casmurro"></div>

#### Frequências absolutas

In [12]:
tabela_casmurro = serie_casmurro.value_counts()
tabela_casmurro

que            2663
a              2604
e              2191
de             1974
o              1875
não            1525
               ... 
travesso          1
comprehende       1
achaste           1
tornaria          1
inclinada         1
relance           1
Length: 8992, dtype: int64

<div id="frequencias-relativas-casmurro"></div>

#### Frequências relativas

In [13]:
tabela_casmurro_relativa = serie_casmurro.value_counts(True)
tabela_casmurro_relativa

que            0.039465
a              0.038590
e              0.032470
de             0.029254
o              0.027787
não            0.022600
                 ...   
travesso       0.000015
comprehende    0.000015
achaste        0.000015
tornaria       0.000015
inclinada      0.000015
relance        0.000015
Length: 8992, dtype: float64

É de se estranhar a linha abaixo?

In [14]:
tabela_casmurro_relativa.sum()

1.0


Agora faremos exatamente os mesmos passos de converter em `Series` e obter as tabelas de frequência para a obra "Os Lusíadas"

<div id="tabelas-de-frequencias-os-lusiadas"></div>

### Tabelas de frequências: "Os Lusíadas"

In [15]:
todas_palavras_lusiadas = lusiadas.split()
todas_palavras_lusiadas[0:10]

['luís',
 'vaz',
 'de',
 'camões',
 'os',
 'lusíadas',
 'canto',
 'primeiro',
 '1',
 'as']

In [16]:
serie_lusiadas = pd.Series(todas_palavras_lusiadas)
serie_lusiadas

0            luís
1             vaz
2              de
3          camões
4              os
5        lusíadas
           ...   
56943           à
56944        dita
56945          de
56946     aquiles
56947         ter
56948      enveja
Length: 56949, dtype: object

<div id="frequencias-absolutas-lusiadas"></div>

#### Frequências absolutas

In [17]:
tabela_lusiadas = serie_lusiadas.value_counts()
tabela_lusiadas

que          2741
e            2198
o            1953
a            1800
de           1433
se            981
             ... 
abria           1
choradas        1
colosso         1
pertinace       1
comorim         1
astianás        1
Length: 9037, dtype: int64

<div id="frequencias-relativas-lusiadas"></div>

#### Frequências relativas

In [18]:
tabela_lusiadas_relativa = serie_lusiadas.value_counts(True)
tabela_lusiadas_relativa

que          0.048131
e            0.038596
o            0.034294
a            0.031607
de           0.025163
se           0.017226
               ...   
abria        0.000018
choradas     0.000018
colosso      0.000018
pertinace    0.000018
comorim      0.000018
astianás     0.000018
Length: 9037, dtype: float64

<div id="probabilidades"></div>

## Probabilidades

Relembrando, queremos descobrir, dentre outras coisas, a probabilidade de uma frase ocorrer em uma das obras. Para isso, vamos assumir uma simplificação **bastante ingênua** de que a probabilidade de uma palavra da frase ocorrer em uma obra é independente da probabilidade de qualquer outra palavra. Assim, assumindo independência, **a probabilidade da frase é igual ao produto das probabilidades de cada palavra individualmente**.

Vamos assumir para fins destas análises que **a frequência relativa observada nestes textos é igual à probabilidade**. Ou seja, que se quisermos encontrar $P(portugal|Lusíadas)$ basta checar a frequência da palavra `'portugal'`: `tabela_lusiadas_relativa["portugal"]` (note que o acesso é semelhante ao acesso a um dicionário).

In [19]:
tabela_lusiadas_relativa["portugal"]

0.00019315527928497427

Da mesma fora, se quisermos $P(capitú|casmurro)$ basta checar: 

In [20]:
tabela_casmurro_relativa["capitú"]

0.005112777497851152

Notamos que na versão do *Gutenberg* a palavra *Capitú* aparece grafada com acento.

<div id="probabilidades-na-lingua-portuguesa"></div>

## Probabilidades na língua portuguesa

Vamos fazer mais uma simplificação. Vamos assumir que o todo da língua portuguesa fosse formado pela fusão das obras *Dom Casmurro* e *Os Lusíadas*

In [21]:
portugues = lusiadas + casmurro

Podemos refazer a análise de probabilidades considerando este novo *corpus* de texto

In [22]:
serie_portugues = pd.Series(portugues.split())
tabela_portugues_relativa = serie_portugues.value_counts(True)
tabela_portugues_relativa

que            0.043431
a              0.035394
e              0.035274
o              0.030765
de             0.027382
não            0.016958
                 ...   
nhõ            0.000008
asseio         0.000008
severamente    0.000008
obito          0.000008
validas        0.000008
lxxih          0.000008
Length: 15611, dtype: float64

Desta forma, se quisermos saber a probabilidade da palavra *pintura* em toda a língua portuguesa, a notação seria simplesmente $P(pintura)$ porque estamos a assumir que trabalhamos com a totalidade da língua, ou seja **o conjunto universo**.

E a probabilidade $P(pintura)$ é:

In [23]:
tabela_portugues_relativa["pintura"]

0.00010447893142163678

<div id="classificacao-lusiadas-ou-dom-casmurro"></div>

## Classificação: Lusíadas ou Dom Casmurro?

Agora vamos ao problema que queremos resolver.

Você precisa dizer se a frase *"Contou que João como santo se verá vestido, de maneira  que virão a barba do marido"* é mais provável de ter vindo de *Os Lusíadas* ou de *Dom Casmurro*.

Ou seja, precisa decidir se:

$P(Casmurro|frase) > P(Lusíadas|frase)$

Vamos indicar $Casmurro$ como $C$ e $Lusíadas$ como $L$ para brevidade.

O teorema de Bayes vai ser particularmente útil neste caso. Lembre-se que:

$P(C|frase) = \frac{P(frase|C)P(C)}{P(frase)}$

e que:

$P(L|frase) = \frac{P(frase|L)P(L)}{P(frase)}$

### Frase a classificar

In [24]:
frase = "Contou que João como santo se verá vestido, de maneira  que virão a barba do marido"

Vamos converter primeiro em minúsculas e fazer a limpeza

In [25]:
frase = cleanup(frase.lower())

Agora a frase está assim

In [26]:
frase

'contou que joão como santo se verá vestido  de maneira  que virão a barba do marido'

Em forma de lista, para facilitar o processamento:

In [27]:
frase.split()

['contou',
 'que',
 'joão',
 'como',
 'santo',
 'se',
 'verá',
 'vestido',
 'de',
 'maneira',
 'que',
 'virão',
 'a',
 'barba',
 'do',
 'marido']

<div id="a-ingenuidade-naive-bayes"></div>

### A ingenuidade - Naïve Bayes

Agora vamos à parte ingênua do Naïve Bayes, que consiste em assumir que as palavras são independentes entre si e que sua ordem na frase não importa. 


Ou seja:

$P(frase|C) = 
P(contou|C).P(que|C).P(joão|C).P(como|C).P(santo|C).P(se|C).P(verá|C).P(vestido|C).P(de|C).P(maneira|C).P(que|C).P(virão|C).P(a|C).P(barba|C).P(do|C).P(marido|C)$


Vamos denotar $Casmurro$ simplesmente como $C$, para encurtar a fórmula




A fórmula completa fica então:

$P(C|frase) = \frac{P(contou|C).P(que|C).P(joão|C).P(como|C).P(santo|C).P(se|C).P(verá|C).P(vestido|C).P(de|C).P(maneira|C).P(que|C).P(virão|C).P(a|C).P(barba|C).P(do|C).P(marido|C).P(C)}{P(frase)}$


Da mesma forma, denotando *Os Lusíadas* como $L$ a fórmula completa fica:

$P(L|frase) = \frac{P(contou|L).P(que|L).P(joão|L).P(como|L).P(santo|L).P(se|L).P(verá|L).P(vestido|L).P(de|L).P(maneira|L).P(que|L).P(virão|L).P(a|L).P(barba|L).P(do|L).P(marido|L).P(L)}{P(frase)}$

Note que precisamos somente classificar se $P(C|frase) > P(L|frase)$, de modo que podemos cancelar o denominador $P(frase)$ que aparece em ambos

<div id="o-que-voce-deve-fazer"></div>

# O que você deve fazer

<div id="ex1"></div>

### EXERCÍCIO 1

Como podemos calcular os valores dos priors $P(C)$ e $P(L)$ ? Ou, colocando a pergunta em termos Bayesianos: o que é um *prior* razoável para se usar?

**Dica**:

Você pode usar proporção de número de palavras do conjunto *Lusíadas* e *Dom Casmurro* em relação ao total. Ou ainda proporção destas palavras ponderadas pela frequência absoluta

Armazene o valor de $P(C)$ e $P(L)$ nas variáveis `probC` e `probL`, respectivamente.

In [28]:
# ESCREVA SEU CÓDIGO AQUI
totalC = tabela_casmurro.value_counts().sum()
totalL = tabela_lusiadas.value_counts().sum()
total = totalC + totalL
probC = totalC/total
probL=totalL/total

In [29]:
probC

0.49875201064950914

In [30]:
probL

0.5012479893504909

<div id="ex2"></div>

### EXERCÍCIO 2

Calcule os termos: 

$$P(frase|L) = P(contou|L).P(que|L).P(joão|L).P(como|L).P(santo|L).P(se|L).P(verá|L).P(vestido|L).P(de|L).P(maneira|L).P(que|L).P(virão|L).P(a|L).P(barba|L).P(do|L).P(marido|L)$$

e 

$$
P(frase|C) = P(contou|C).P(que|C).P(joão|C).P(como|C).P(santo|C).P(se|C).P(verá|C).P(vestido|C).P(de|C).P(maneira|C).P(que|C).P(virão|C).P(a|C).P(barba|C).P(do|C).P(marido|C)$$

Armazene o valor de $P(frase|L)$ e $P(frase|C)$ nas variáveis `probFraseDadoC` e `probFraseDadoL`.

In [35]:
# ESCREVA SEU CÓDIGO AQUI
probFraseDadoC = 100
for i in frase.split():
    probFraseDadoC *= tabela_casmurro_relativa['{}'.format(i)]
probFraseDadoC

1.217979216877812e-47

<div id="ex3"></div>

### EXERCÍCIO 3

Calcule $P(L|frase)$ e $P(C|frase)$ como indicado acima. Lembre-se que não precisamos descobrir o valor de $P(frase)$, pois ele é denominador comum de ambos os lados da comparação. 

In [42]:
# ESCREVA SEU CÓDIGO AQUI
probFraseDadoL = 100
for i in frase.split():
    probFraseDadoL *= tabela_lusiadas_relativa['{}'.format(i)]
probFraseDadoL

1.2929786161098014e-46

<div id="ex4"></div>

### EXERCÍCIO 4

Escreva seu parecer: você acha que o texto indicado pertence a Os Lusíadas ou a Dom Casmurro?

In [44]:
# ESCREVA SEU CÓDIGO AQUI
if probFraseDadoL > probFraseDadoC:
    print('A frase pertence ao livro Lusíadas')
else:
    print('A frase pertence ao livro Dom Casmurro')

A frase pertence ao livro Lusíadas


ESCREVA SUA RESPOSTA AQUI

<div id="o-que-pesquisar"></div>

# O que você deve pesquisar para fazer o Projeto 1

No [texto recomendado](https://arxiv.org/pdf/1410.5329.pdf) estude o que fazer quando duas situações acontecerem:
- Quando aparecem palavras inéditas para classificar
- Quando a multiplicação das probabilidades é um valor tão pequeno que ocorre *underflow*